# 付録 10.4.2: Claudeを使って文書を転写する方法

Claude 3は、非構造化テキストや画像およびPDF内の情報を読み取り、それを構造化テキストに変換するのが得意です。いくつかの例を見ていきますが、その前にノートブックを実行するために必要なコードを設定しましょう。

In [ ]:
pip install -qUr requirements.txt

In [ ]:
import boto3
import json
from datetime import datetime
from IPython.display import Image
from botocore.exceptions import ClientError

session = boto3.Session()
region = session.region_name

In [ ]:
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
#modelId = 'anthropic.claude-3-haiku-20240307-v1:0'

print(f'Using modelId: {modelId}')
print('使用しているリージョン: ', region)

bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = region,)

## タイプされたテキストの転写

従来のOCRシステムに対するClaude 3の利点は、Claude 3の高度な推論能力により、転写したい内容を正確に指定できることです。この画像については、回答の中のコードだけを転写しましょう。

In [ ]:
from IPython.display import Image
Image(filename='./images/transcribe/stack_overflow.png')

In [ ]:
with open("./images/transcribe/stack_overflow.png", "rb") as f:
    image_file = f.read()

messages = [
    {
        "role": 'user',
        "content": [
            {"text": "回答のコードを転写してください。コードのみ出力します。"},
            {"image": {
                "format": 'png',
                "source": {"bytes": image_file }
                },
            }
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
}

response = bedrock_client.converse(**converse_api_params)

# レスポンスから生成されたテキストコンテンツを抽出します
output_message = response['output']['message']['content'][0]['text']

# 生成されたテキストコンテンツを返します
print(output_message)

## 手書きテキストの転写

それは良いですが、もう少し難しいことを試してみましょう。`Claude 3`は手書きのテキストの転写にも優れています。`Claude 3`にこの手書きの処方箋メモを転写するように頼んでみましょう。

In [ ]:
Image(filename='./images/transcribe/school_notes.png')

In [ ]:
with open("./images/transcribe/school_notes.png", "rb") as f:
    image_file = f.read()

messages = [
    {
        "role": 'user',
        "content": [
            {"text": "このテキストを転写してください。テキストのみを出力し、他には何も出力しないでください。"},
            {"image": {
                "format": 'png',
                "source": {"bytes": image_file }
                },
            }
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
}

response = bedrock_client.converse(**converse_api_params)

# レスポンスから生成されたテキストコンテンツを抽出します
output_message = response['output']['message']['content'][0]['text']

# 生成されたテキストコンテンツを返します
print(output_message)

## フォームの転写
タイプされたテキストと手書きのテキストの組み合わせを試してみませんか？これは、保険や報告書のフォームなど、さまざまな文書で一般的です。

In [ ]:
Image(filename='./images/transcribe/vehicle_form.jpg')

In [ ]:
with open("./images/transcribe/vehicle_form.jpg", "rb") as f:
    image_file = f.read()

messages = [
    {
        "role": 'user',
        "content": [
            {"text": "このフォームを正確に転写してください。"},
            {"image": {
                "format": 'jpeg',
                "source": {"bytes": image_file }
                },
            }
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
}

response = bedrock_client.converse(**converse_api_params)

# レスポンスから生成されたテキストコンテンツを抽出します
output_message = response['output']['message']['content'][0]['text']

# 生成されたテキストコンテンツを返します
print(output_message)

## 複雑な文書QA  
Claude 3を使用すると、単なる転写を超えて、非構造化文書内の情報に関して具体的な質問をすることができます。

In [ ]:
Image(filename='./images/transcribe/page.jpeg')

In [ ]:
with open("./images/transcribe/page.jpeg", "rb") as f:
    image_file = f.read()

messages = [
    {
        "role": 'user',
        "content": [
            {"text": "Which is the most critical issue for live rep support?"},
            {"image": {
                "format": 'jpeg',
                "source": {"bytes": image_file }
                },
            }
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
}

response = bedrock_client.converse(**converse_api_params)

# レスポンスから生成されたテキストコンテンツを抽出します
output_message = response['output']['message']['content'][0]['text']

# 生成されたテキストコンテンツを返します
print(output_message)

## 非構造情報 -> JSON

画像内の非構造情報を構造化されたJSON出力に変換するためにClaudeをどのように使用できるか見てみましょう。

In [ ]:
Image(filename='./images/transcribe/org_chart.jpeg')

In [ ]:
with open("./images/transcribe/org_chart.jpeg", "rb") as f:
    image_file = f.read()

messages = [
    {
        "role": 'user',
        "content": [
            {"text": "この組織図をJSONに変換して、誰が誰に報告しているかを示してください。JSONのみを出力し、それ以外は出力しないでください。"},
            {"image": {
                "format": 'jpeg',
                "source": {"bytes": image_file }
                },
            }
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
}

response = bedrock_client.converse(**converse_api_params)

# レスポンスから生成されたテキストコンテンツを抽出する
output_message = response['output']['message']['content'][0]['text']

# 生成されたテキストコンテンツを返す
print(output_message)